In [154]:
import ftplib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce
import paramiko 
import datetime
from datetime import date

In [155]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)
paramiko.util.log_to_file("paramiko.log")

In [156]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy)==1:
    mes_hoy = "0"+mes_hoy
if len(dia_hoy)==1:
    dia_hoy = "0"+dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost =  "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost   

#Direccion descarga local
direccionDescargaLocal = "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost 
    
print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2023-08-31
31082023
VentaCARE_ASSISTANCE_31082023.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_31082023.xlsx


In [157]:
host="sftp.bciseguros.cl"
puerto=22
transporte = paramiko.Transport((host,puerto))
usuario= "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None,usuario,clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo

In [158]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath,localpath)


In [159]:
if sftp: sftp.close()
if transporte: transporte.close()

Inicio Manipulacion de Excel descargado

In [160]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(2357, 17)

In [161]:
archivoBCI.dtypes

POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [162]:
archivoBCI.RUT_ASEGURADO.fillna("0", inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)

archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2597777,2023-04-25,2024-04-24,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,100,7,100,7,MASCULINO,TITULAR MASCULINO,JOSE R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0
1,2597778,2023-05-03,2024-05-02,SAIN,M7,SALUD INDIVIDUAL PLAN 60 RECOVER,100,7,100,7,MASCULINO,TITULAR MASCULINO,JOSE R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0
2,2597779,2023-05-03,2024-05-02,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,100,7,100,7,MASCULINO,TITULAR MASCULINO,JOSE R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0
3,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15583606,7,MASCULINO,TITULAR MASCULINO,JESUS NOE REY ACEVEDO,1983-02-03,contaservicoser@gmail.com,SANTIAGO,933111465
4,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15797990,6,FEMENINO,ESPOSA,Gajardo Pradenas Cynthia Del Pilar,1984-04-02,cgajardo34@gmail.com,SANTIAGO,996766488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,2643884,2023-08-30,2024-08-29,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16715039,K,16715039,K,FEMENINO,TITULAR FEMENINO,JASMIN ANDREA ARIAS LEIVA,1987-08-14,yo_ch_y20@hotmail.com,CALLE BOLIVIA 2501,991815552
2303,2643875,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,19515712,K,19515712,K,MASCULINO,TITULAR MASCULINO,DEMIAN NICOLAS GUTIERREZ ACUÑA,1996-09-28,SANDRA.ACUNA2@GMAIL.COM,NO INFORMADO,0
2304,2643876,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,26603860,7,26603860,7,FEMENINO,TITULAR FEMENINO,FUENMAYOR MONSALVE MARY MAURA,1981-02-09,marymaurafuenmayor@gmail.com,SANTIAGO,935788822
2305,2643876,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,26603860,7,26603763,5,FEMENINO,HIJA,JENMAR ELENA MARTINEZ FUENMAYOR,2007-10-26,NO INFORMADO,NO INFORMADO,0


In [163]:
archivoBCI["Rut Titular"] = (
    archivoBCI["RUT_TITULAR"].astype(str) + "-" + archivoBCI["DV_TITULAR"].astype(str)
)
archivoBCI["Rut Carga"] = (
    archivoBCI["RUT_ASEGURADO"].astype(str)
    + "-"
    + archivoBCI["DV_ASEGURADO"].astype(str)
)
general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion ={
    "no informado": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str)
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("0", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")



C:\Users\jmartinez\AppData\Local\Temp\ipykernel_2168\3684553287.py:56: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [164]:
archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR,Rut Titular,Rut Carga,email,direccion,celular
0,2597777,2023-04-25,2024-04-24,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,100,7,100,7,...,JOSE R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0,100-7,100-7,paulitalorena@hotmail.com,Pasaje Manco Capac Nro 1745,
1,2597778,2023-05-03,2024-05-02,SAIN,M7,SALUD INDIVIDUAL PLAN 60 RECOVER,100,7,100,7,...,JOSE R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0,100-7,100-7,paulitalorena@hotmail.com,Pasaje Manco Capac Nro 1745,
2,2597779,2023-05-03,2024-05-02,SAIN,M8,SALUD INDIVIDUAL PLAN 70 RECOVER,100,7,100,7,...,JOSE R FRIAZ RUIZ,1991-01-01,PAULITALORENA@HOTMAIL.COM,PASAJE MANCO CAPAC NRO 1745,0,100-7,100-7,paulitalorena@hotmail.com,Pasaje Manco Capac Nro 1745,
3,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15583606,7,...,JESUS NOE REY ACEVEDO,1983-02-03,contaservicoser@gmail.com,SANTIAGO,933111465,15583606-7,15583606-7,contaservicoser@gmail.com,Santiago,933111465
4,2601791,2023-06-01,2024-05-31,SAIN,M9,SALUD INDIVIDUAL PLAN 80 RECOVER,15583606,7,15797990,6,...,Gajardo Pradenas Cynthia Del Pilar,1984-04-02,cgajardo34@gmail.com,SANTIAGO,996766488,15583606-7,15797990-6,cgajardo34@gmail.com,Santiago,996766488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,2643884,2023-08-30,2024-08-29,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16715039,K,16715039,K,...,JASMIN ANDREA ARIAS LEIVA,1987-08-14,yo_ch_y20@hotmail.com,CALLE BOLIVIA 2501,991815552,16715039-K,16715039-K,yo_ch_y20@hotmail.com,Calle Bolivia 2501,991815552
2303,2643875,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,19515712,K,19515712,K,...,DEMIAN NICOLAS GUTIERREZ ACUÑA,1996-09-28,SANDRA.ACUNA2@GMAIL.COM,NO INFORMADO,0,19515712-K,19515712-K,sandra.acuna2@gmail.com,,
2304,2643876,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,26603860,7,26603860,7,...,FUENMAYOR MONSALVE MARY MAURA,1981-02-09,marymaurafuenmayor@gmail.com,SANTIAGO,935788822,26603860-7,26603860-7,marymaurafuenmayor@gmail.com,Santiago,935788822
2305,2643876,2023-08-30,2024-08-29,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,26603860,7,26603763,5,...,JENMAR ELENA MARTINEZ FUENMAYOR,2007-10-26,NO INFORMADO,NO INFORMADO,0,26603860-7,26603763-5,,,


In [165]:
#archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
